## Imports

In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

from transformers import BertTokenizer
from transformers import BertModel

## Load BERT Model from PT File

In [3]:
if torch.has_mps:
    device = 'mps'
else:
    device = 'cpu'

print(device)

mps


In [34]:
train_data_path = 'Data/BF_Single_Subject_Clean_Call_Number.csv'
df = pd.read_csv(train_data_path, sep='\t')
df = df.drop("Unnamed: 0", axis=1)



# Hardcoded fix
for i, row in df.iterrows():
  if row['Call_Number'] == '724.55.C63':
    print("Trash")
    df.at[i, 'Call_Number'] = '724'


vals = pd.unique(df['Call_Number'])
print(len(vals))
labels = {key: value for (key, value) in enumerate(vals)}

Trash
657


In [35]:
print(labels)

{0: '174', 1: '1583', 2: '76.5', 3: '636', 4: '175.5', 5: '121', 6: '723', 7: '1598', 8: '210', 9: '870', 10: '711', 11: '698.5', 12: '201', 13: '323', 14: '311', 15: '713', 16: '319', 17: '431', 18: '176', 19: '637', 20: '18.02', 21: '175', 22: '633', 23: '632.5', 24: '1576', 25: '173', 26: '710', 27: '698', 28: '378', 29: '697', 30: '1325', 31: '408', 32: '432', 33: '113', 34: '325', 35: '448', 36: '639', 37: '295', 38: '371', 39: '207', 40: '721', 41: '1593', 42: '692.2', 43: '1407', 44: '724', 45: '575', 46: '318', 47: '1779', 48: '341', 49: '1999', 50: '692.5', 51: '162', 52: '39', 53: '321', 54: '441', 55: '38', 56: '714', 57: '1622', 58: '447', 59: '109', 60: '76.4', 61: '1791', 62: '241', 63: '1611', 64: '698.3', 65: '316.6', 66: '1561', 67: '105', 68: '315', 69: '701', 70: '335', 71: '698.95', 72: '455', 73: '1793', 74: '201.3', 75: '38.5', 76: '724.3', 77: '1434', 78: '233', 79: '774', 80: '720', 81: '717', 82: '458', 83: '531', 84: '81', 85: '31', 86: '1591', 87: '39.9', 88:

In [7]:
print(vals)

['174' '1583' '76.5' '636' '175.5' '121' '723' '1598' '210' '870' '711'
 '698.5' '201' '323' '311' '713' '319' '431' '176' '637' '18.02' '175'
 '633' '632.5' '1576' '173' '710' '698' '378' '697' '1325' '408' '432'
 '113' '325' '448' '639' '295' '371' '207' '721' '1593' '692.2' '1407'
 '724' '575' '318' '1779' '341' '1999' '692.5' '162' '39' '321' '441' '38'
 '714' '1622' '447' '109' '76.4' '1791' '241' '1611' '698.3' '316.6'
 '1561' '105' '315' '701' '335' '698.95' '455' '1793' '201.3' '38.5'
 '724.3' '1434' '233' '774' '720' '717' '458' '531' '81' '31' '1591'
 '39.9' '1241' '1714' '698.8' '318.5' '692' '503' '18.022' '412' '1031'
 '1141' '698.4' '1584' '346' '1261.2' '511' '1242' '1729' '1623' '1701'
 '671' '242' '319.5' '171' '1559' '175.4' '1556' '1472' '1534' '1028.5'
 '204' '449' '1520' '724.5' '367' '683' '161' '1581' '468' '199' '1567'
 '1566' '722' '697.5' '1301' '293' '77' '611' '416' '1078' '445' '444'
 '209' '1042' '125' '1671' '353' '1548' '456' '1275' '108' '1615' '1770'
 

In [5]:
class BertClassifier(nn.Module):
  def __init__(self, dropout=0.5):
    super(BertClassifier, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, len(vals)) # Change depending on dataset size
    self.relu = nn.ReLU()

  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)
    final_layer = self.relu(linear_output)

    return final_layer

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [19]:
model_path = "Models/BERT_Classifier_25_epochs_subjects_clean.pt"

model = BertClassifier()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## Load EBook Data

In [11]:
## WIP There are various encoding errors in the dataset, maybe need to hand scrape myself

ebook_path = 'Data/EbookListforALA.csv'
ebook_df = pd.read_csv(ebook_path)
ebook_df = df.drop("Unnamed: 0", axis=1)
ebook_df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 1385: invalid continuation byte

In [18]:
header_in = 'Consciousness' #,e001,,"Trauma and its impacts on temporal experience New perspectives from phenomenology and psychoanalysis","Mezzalira, Selene",2022,"Psychic trauma, Time perception, Consciousness",https://www.syndetics.com/index.php?isbn=100053166X/lc.jpg&client=sirsi,https://search.lib.byu.edu/byu/record/cat.7418121.item.7418121-1001?holding=4z8xrtkcbglsff40
headers_in = "Psychic trauma, Time perception, Consciousness"

## Predict output and assign label

In [23]:
#model.to(device)
# TODO: Put this into a loop for each header from ebook dataset. Write output back to dataframe.

tokenized = tokenizer(headers_in, padding='max_length',max_length=512, truncation=True,return_tensors="pt")
input_id = tokenized['input_ids']
mask = tokenized['attention_mask']

In [25]:
output = model(input_id, mask)
print(output.shape)

torch.Size([1, 657])


In [37]:
res = output.argmax(dim=1)
print(labels[res.item()])

109


In [ ]:
## TODO: Map to subject header gropus just for reference? someway to check if it looks right ish?